In [ ]:
!pip install pandas matplotlib seaborn --quiet
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests

sns.set(style='darkgrid')

In [ ]:
# Appel à l'API OpenSky : données temps réel
url = "https://opensky-network.org/api/states/all"

# 17 colonnes de l’API OpenSky
columns = [
    "icao24", "callsign", "origin_country", "time_position", "last_contact",
    "longitude", "latitude", "baro_altitude", "on_ground", "velocity",
    "true_track", "vertical_rate", "sensors", "geo_altitude",
    "squawk", "spi", "position_source"
]


In [ ]:
# Appel de l'api
response = requests.get(url)
data = response.json() # Conversion en JSON ---> Python
data.keys()

dict_keys(['time', 'states'])

In [ ]:
df = pd.DataFrame(data['states'], columns=columns)
df.head()

,icao24,callsign,origin_country,time_position,last_contact,longitude,latitude,baro_altitude,on_ground,velocity,true_track,vertical_rate,sensors,geo_altitude,squawk,spi,position_source
0,4b1814,SWR8CD,Switzerland,1.765537e+09,1765536740,7.4198,47.6509,5379.72,False,223.85,108.35,-15.28,None,5585.46,2027,False,0
1,39de4e,TVF78YY,France,1.765537e+09,1765536740,-7.4689,43.3675,10652.76,False,256.50,41.75,5.20,None,10706.10,None,False,0
2,4b1819,SWR5ZM,Switzerland,1.765537e+09,1765536740,16.7767,58.1202,8816.34,False,215.20,32.70,-15.28,None,8900.16,3010,False,0
3,e8027b,LAN500,Chile,1.765537e+09,1765536740,-80.7668,25.7396,1905.00,False,128.53,11.31,-4.55,None,1988.82,None,False,0
4,80162a,IGO7309,India,1.765537e+09,1765536740,90.4203,23.2146,5486.40,False,129.40,253.37,0.00,None,5654.04,2601,False,0


In [ ]:
# Nettoayge minimal
# Supprimer les avions sans coordonnées GPS

df = df.dropna(subset=['latitude', 'longitude'])
df.head()

# Convertir la vitesse en Km/h (API = m/s)
df["velocity_kmh"] = df["velocity"] * 3.6
df.head()

,icao24,callsign,origin_country,time_position,last_contact,longitude,latitude,baro_altitude,on_ground,velocity,true_track,vertical_rate,sensors,geo_altitude,squawk,spi,position_source,velocity_kmh
0,4b1814,SWR8CD,Switzerland,1.765537e+09,1765536740,7.4198,47.6509,5379.72,False,223.85,108.35,-15.28,None,5585.46,2027,False,0,805.860
1,39de4e,TVF78YY,France,1.765537e+09,1765536740,-7.4689,43.3675,10652.76,False,256.50,41.75,5.20,None,10706.10,None,False,0,923.400
2,4b1819,SWR5ZM,Switzerland,1.765537e+09,1765536740,16.7767,58.1202,8816.34,False,215.20,32.70,-15.28,None,8900.16,3010,False,0,774.720
3,e8027b,LAN500,Chile,1.765537e+09,1765536740,-80.7668,25.7396,1905.00,False,128.53,11.31,-4.55,None,1988.82,None,False,0,462.708
4,80162a,IGO7309,India,1.765537e+09,1765536740,90.4203,23.2146,5486.40,False,129.40,253.37,0.00,None,5654.04,2601,False,0,465.840


In [ ]:
# Statistique globale du trafic aériens
print("Nombre total d'avions actuellement en vole : ", len(df))
print("Altitude moyenne des avions en vole : ", df["geo_altitude"].mean(),)
print("Vitesse moyenne des avions en vole : ", df["velocity_kmh"].mean(), "km/h")
print("Top 10 des pays d'origine des avions en vole :")
df["origin_country"].value_counts().head(10)


Nombre total d'avions actuellement en vole :  5939
Altitude moyenne des avions en vole :  7736.429160045403
Vitesse moyenne des avions en vole :  612.8505772987538 km/h
Top 10 des pays d'origine des avions en vole :


,count
origin_country,
United States,1015
United Kingdom,421
Germany,284
Ireland,266
Turkey,245
China,240
France,204
India,195
United Arab Emirates,181
